In [2]:
import pickle 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [6]:
# importing the model , scaler and gender_label_encoder as well as onehot_encoder_geo
model = load_model('model.h5')
with open('churn_modelling_scaler.pkl' , 'rb') as file:
    scaler = pickle.load(file)
with open('gender_label_encoder.pkl' ,'rb') as file:
    gender_label_encoder = pickle.load(file)
with open('onehot_encoder_geo.pkl' , 'rb') as file:
    onehot_encoder_geo = pickle.load(file)


In [7]:
input_data = {
    'CreditScore' : 600 , 
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance' : 60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'Geography':'France',
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [10]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded , columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\gm782\OneDrive\Desktop\mauravgalik\MLUsingLangchain\ANN_ClassificationProject\classificationEnv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,Geography,IsActiveMember,EstimatedSalary
0,600,Male,40,3,60000,2,1,France,1,50000


In [16]:
input_data = pd.concat([input_df.reset_index(drop = True) , geo_encoded_df] , axis=1)

In [17]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,Geography,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,France,1,50000,1.0,0.0,0.0


In [18]:
input_data['Gender'] = gender_label_encoder.transform(input_data['Gender'])
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,Geography,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,France,1,50000,1.0,0.0,0.0


In [20]:
input_data.drop(['Geography'] , axis=1 , inplace=True)

In [21]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [22]:
input_scaled = scaler.transform(input_data)

In [23]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [24]:
prediction = model.predict(input_scaled)
prediction_proba = prediction[0][0]
print(prediction)
print(prediction_proba)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
[[0.02235254]]
0.02235254


In [25]:
if prediction_proba > 0.5:
    print("The person is likely to churn.")
else:
    print("The person is not likely to churn,")

The person is not likely to churn,
